# Training

In [1]:
import pickle
with open('train.dat', 'rb') as outfile:
    save_obj = pickle.load(outfile)

In [2]:
contexts, questions, answers, vocab_size = \
save_obj['contexts'].toarray(),\
save_obj['questions'].toarray(),\
save_obj['answers'].toarray(),\
save_obj['vocab_size']

In [43]:
from keras.layers import Input, Embedding, LSTM, Dense, merge
from keras.layers import TimeDistributed
from keras.models import Model

def build_model(context_len, question_len, vocab_size, embed_dim=100):
    context_input = Input(shape=(context_len,), dtype='int32', name='context_input')
    
    context_embedding = Embedding(output_dim=embed_dim, input_dim=vocab_size, dropout=0.3)(context_input)
    # output: (samples, context_len, embedding_dim)
        
    question_input = Input(shape=(question_len,), dtype='int32', name='question_input')
    question_embedding = Embedding(output_dim=embed_dim, input_dim=vocab_size, dropout=0.3)(question_input)
    # output: (samples, question_len, embedding_dim)
    
    merged = merge([context_embedding, question_embedding], mode='dot', dot_axes=[2, 2])
    # output: (samples, context_len, question_len)
    
    lstm_merge = LSTM(128, return_sequences=True)(merged)
    # output: (samples, context_len, 128)
    
    predictions = TimeDistributed(Dense(2, activation='softmax'))(lstm_merge)
    
    model = Model(input=[context_input, question_input], output=predictions)
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
    return model

model = build_model(contexts.shape[1], questions.shape[1], vocab_size, embed_dim=100)

In [10]:
answers = answers.astype('int32')

In [12]:
from keras.utils import np_utils
rows, cols= answers.shape
answers = np_utils.to_categorical(answers.reshape((rows*cols,)), nb_classes = 2).reshape((rows, cols, -1))

In [32]:
print(answers[0].shape)
print(contexts[0].shape)

(3706, 2)
(3706,)


In [ ]:
model.fit([contexts, questions], answers, nb_epoch=50, batch_size=32)

Epoch 1/50
